In [3]:
# read in imports
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.cross_validation import cross_val_score

In [4]:
# set up folder link and files
folder_link = 'C:/Users/Michael/projects/kaggle/rossman/'

train_file = folder_link + 'train.csv'
test_file = folder_link + 'test.csv'
submission_file = folder_link + 'predictions_Ensemble.csv'
store_file = folder_link + 'store.csv'

In [5]:
# set up dataframes
train = pd.read_csv(train_file)
test = pd.read_csv(test_file)
store = pd.read_csv(store_file)

C:\Users\Michael\Anaconda\lib\site-packages\pandas\io\parsers.py:1170: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [6]:
# merge train and store data
train1 = train

# change state holiday to numeric
train1.StateHoliday = train1.StateHoliday.map({0:0,'0':1,'a':2,'b':3,'c':4})
train1.StateHoliday = train1.StateHoliday.astype(int)

In [73]:
# create features and response
feature_cols = ['DayOfWeek','Store','Open','Promo','StateHoliday','SchoolHoliday']
train2 = train1.sample(n=1000)
X = train2[feature_cols]
y = train2[['Sales','Open']]

In [8]:
# create three samples
n_rows = X.shape[0]

# set a seed for reproducibility
np.random.seed(1234)

#create three bootstrap samples
sample1 = np.random.choice(a=n_rows,size=n_rows,replace=True)
sample2 = np.random.choice(a=n_rows,size=n_rows,replace=True)
sample3 = np.random.choice(a=n_rows,size=n_rows,replace=True)

In [38]:
sample1

array([815, 723, 294,  53, 204, 372, 664, 655, 689, 279, 233, 154,  30,
       683, 158, 748, 892, 794, 374, 442, 120, 880, 476, 325, 336, 969,
       383, 246, 510, 431, 562, 752, 763,  76, 293, 802, 934, 506, 195,
       743, 139, 640, 745, 843, 336,   3, 258, 275, 908, 961, 459, 371,
       619, 465,  14, 243, 455, 828, 490, 619, 942, 924,  81, 855, 781,
       116, 608, 908, 197, 625, 991, 997, 671, 761, 601,  84,  45, 528,
        41, 840, 440, 236, 838, 952,  86, 940, 986, 979, 303, 689,  18,
       597, 686, 482, 805, 365, 806, 135, 996, 707, 901, 750, 559, 244,
       374, 687, 911, 546,  10, 284,   4,  82, 103, 345, 311, 235, 334,
       663, 434, 761, 574, 183, 340, 617, 256, 633, 354, 761, 346, 417,
       882, 377, 497, 789, 967, 196, 119, 999, 465, 250, 436, 448, 469,
       372, 873, 626, 425,   1, 142, 771, 798, 396, 713, 877, 787,  26,
       992,  68, 891, 996, 832, 150, 440, 980, 136, 684, 753, 920, 990,
       745, 527, 840, 898, 400, 619, 130, 754, 591, 195,  46, 48

In [9]:
# remove NANs from test
test.loc[test.Open.isnull(),'Open'] = 1
test.loc[test.StateHoliday.isnull(),'StateHoliday'] = 0
test.StateHoliday = test.StateHoliday.map({0:0,'0':1,'a':2,'b':3,'c':4})
test.StateHoliday = test.StateHoliday.astype(int)

In [91]:
# sample
y.iloc[sample1,0]

225206        0
800012        0
577561     2650
66146     13449
659990        0
505430     5834
274548     4449
642053    10992
642927    18862
537315     5752
688263     4146
756850     5128
172575     6645
527078        0
786442     3949
376981        0
26808      5594
641586     7959
920424     6833
883805    14959
117996     9212
197150     9687
796239     5796
14483         0
512750     4739
448224     5513
959881     6878
32822      3621
855420        0
875408     5350
          ...  
575680     8524
260067     5235
67111      7022
933772     4629
34108      8151
540212     8847
480605        0
594137     4820
7390       5933
113265     5885
233423     7893
578843     5639
837699     4745
554766     4901
177915        0
97065      4881
89276      5726
769444        0
853351     9043
767353    11018
888224     8792
707313     4717
360063     6244
583456     7537
980356        0
872368     6914
896098     4590
363663        0
91669         0
129826        0
Name: Sales, dtype: int6

In [92]:
# grow one regression tree with each bootstrapped training set
treereg1 = DecisionTreeRegressor(max_depth=1,random_state=1234)
treereg1.fit(X.iloc[sample1,:],y.iloc[sample1,0])

treereg2 = DecisionTreeRegressor(max_depth=1,random_state=1234)
treereg2.fit(X.iloc[sample2,:],y.iloc[sample2,0])

treereg3 = DecisionTreeRegressor(max_depth=1,random_state=1234)
treereg3.fit(X.iloc[sample3,:],y.iloc[sample3,0])



DecisionTreeRegressor(criterion='mse', max_depth=1, max_features=None,
           max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, random_state=1234,
           splitter='best')

In [93]:
# make predictions with each bootstrapped training set
preds1 = treereg1.predict(test[feature_cols])
preds2 = treereg2.predict(test[feature_cols])
preds3 = treereg3.predict(test[feature_cols])

In [98]:
y_pred = (preds1 + preds2 + preds3) / 3

In [99]:
# create submission dataframe
submit = pd.DataFrame({'Id':test['Id'],'Sales':y_pred})

In [100]:
# create csv file
submit.to_csv(submission_file,index=False)